In [ ]:
        # Read DF, convert columns to right dtype and filter by `train` and `fold`
        df = pd.read_csv(os.path.join(root, f"merged_datasets_{size}.csv"))
        df = df.astype({"target": int, "fold": str})
        if train:
            df = df[(df["fold"] != str(fold)) & (df["fold"] != "test") & (df["dataset"].isin(datasets))]
        else:
            df = df[(df["fold"] == str(fold)) & (df["dataset"].isin(datasets))]
            

# Imports

In [1]:
# General imports
import os
import sys
import random 
import functools

import cv2
import piq
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm
import matplotlib.pyplot as plt 
import albumentations as albu
import albumentations.pytorch as albu_pt

In [2]:
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

# Fix to be able to import python modules inside a notebook
os.chdir('..')

# Useful extensions
# %load_ext watermark
# %watermark -v -n -m -p numpy,torch,albumentations,photosynthesis_metrics

# %load_ext autoreload
# %autoreload 2

# Nice plot formating
%matplotlib inline

In [4]:
!ls

configs   logs	    notebooks  reports		 test_images.zip  WORKPLAN.md
data	  Makefile  old_logs   requirements.txt  tests
labeling  models    README.md  src		 train.py


Training GAN's experiments:

GAN runner
GAN callback to support custom metrics

GAN model (from Skoltech DL course)



# Parse model results to get best checkpoints

In [43]:
from src.features.models import MODEL_FROM_NAME

def get_best_model(task: str, metric: str, mode: str = "min", root: str = "logs") -> nn.Module:
    """Returns best model for specific task and metric
    Args:
        task: One of {`sr`, `denoise`, `deblur`}
        metric: Metric name
        mode: Find minimum or maximum value
        root: Path to folder with logs
    """
    task_folders = []
    for folder_name in os.listdir(root):
        if task == folder_name.split("_")[0]:
            task_folders.append(folder_name)
            
    if mode == "min":
        best = np.inf
        monitor_op = np.less
    elif mode == "max":
        best = - np.inf
        monitor_op = np.greater
    else:
        raise ValueError(f"Mode {mode} not defined!")
    
    model_name, model_weights = None, None

    for folder in task_folders:
        # Load checkpoint
        checkpoint_path = os.path.join(root, folder, f"model_{metric}.chpn")
        checkpoint = torch.load(checkpoint_path)
        current = checkpoint["value"]
        if monitor_op(current, best):
            best = current
            model_name = folder.split("_")[2]
            model_weights = checkpoint["state_dict"]
        
    # Load model
    model = MODEL_FROM_NAME[model_name]()
    model.load_state_dict(model_weights)
    print(f"Found model with best {metric} value {best:.4f}")
    return model
       
model = get_best_model(task="deblur", metric="psnr", mode="max")

Found model with best psnr value 25.5789


In [63]:
input = torch.rand(1,3, 288, 128)
model(input)

tensor([[[[0.4999, 0.5428, 0.7437,  ..., 0.6626, 0.4843, 0.5072],
          [0.4708, 0.4690, 0.6330,  ..., 0.6996, 0.6185, 0.5259],
          [0.4090, 0.3908, 0.6269,  ..., 0.7073, 0.6420, 0.5910],
          ...,
          [0.7245, 0.4985, 0.4155,  ..., 0.2828, 0.2729, 0.4264],
          [0.4935, 0.7046, 0.4990,  ..., 0.4395, 0.4466, 0.4762],
          [0.4798, 0.6257, 0.6719,  ..., 0.6877, 0.6423, 0.4916]],

         [[0.4895, 0.4619, 0.4943,  ..., 0.8595, 0.6318, 0.7945],
          [0.4380, 0.4043, 0.4516,  ..., 0.8498, 0.7570, 0.7280],
          [0.4426, 0.3638, 0.5839,  ..., 0.7171, 0.6137, 0.7215],
          ...,
          [0.8938, 0.4837, 0.3751,  ..., 0.2228, 0.2042, 0.3832],
          [0.5795, 0.5868, 0.4542,  ..., 0.3856, 0.3783, 0.4453],
          [0.4976, 0.6766, 0.6273,  ..., 0.4947, 0.6034, 0.4811]],

         [[0.7272, 0.4917, 0.6660,  ..., 0.7403, 0.6129, 0.4999],
          [0.7777, 0.4782, 0.4875,  ..., 0.7840, 0.5016, 0.4995],
          [0.4947, 0.4366, 0.8937,  ..., 0

# Inference images into separate folder

In [17]:

f"{i:04d}"

'0123'

In [24]:
import cv2



In [5]:
# Inference selected images
from src.data.datasets import DIV2K
from PIL import Image

# Images used in paper `Comparison of Image Quality Models for Optimization of Image Processing Systems`
INDICES = [3, 5, 8, 11, 18, 21, 22, 24, 34, 49, 54, 55, 60, 64, 68, 
#            69, 82, 83, 88
          ]

@torch.no_grad()
def inference_model(task: str, metric: str, mode: str = "min", root: str = "logs"):
    
    # Create output folder
    output_path = f"data/processed/{task}/{metric}"
    os.makedirs(output_path, exist_ok=True)
    
    # Find best model
    model = get_best_model(task, metric, mode, root).cuda()
    
    # Get augmentations
    NORM_TO_TENSOR = albu.Compose([
        albu.Normalize(mean=[0., 0., 0.], std=[1., 1., 1.], ), # to [0, 1]
        albu_pt.ToTensorV2()],
        additional_targets={"mask": "image"})
    
    if task == "deblur":
        TASK_AUG = albu.OneOf([
            albu.Blur(blur_limit=(3, 5)),
            albu.GaussianBlur(blur_limit=(3, 5)),
        ], p=1.0)
    elif task == "denoise":
        TASK_AUG = albu.OneOf([
            albu.MultiplicativeNoise(multiplier=(0.75, 1.25), per_channel=True, elementwise=True),
        ], p=1.0)
    elif task == "sr":
        TASK_AUG = albu.Downscale(
            scale_min=0.5, scale_max=0.5, interpolation=cv2.INTER_CUBIC, always_apply=True)
        
    transform = albu.Compose([
        TASK_AUG,
        NORM_TO_TENSOR,
    ])
    
    dataset = DIV2K(train=False, transform=transform)
    
    for i, idx in enumerate(INDICES):
        input, target = dataset[idx]
        
        # Add batch dimension
        input, target = input.unsqueeze(0).cuda(), target.unsqueeze(0).cuda()
        _, _, H, W = input.shape

        # It's important for Unet to have image with size dividible by 32, so bad image manuallu and then crop
        H_pad, W_pad = (32 - H % 32) % 32, (32 - W % 32) % 32
        
        input = torch.nn.functional.pad(input, pad=(0, W_pad, 0, H_pad))
        target = torch.nn.functional.pad(input, pad=(0, W_pad, 0, H_pad))
#         print(input.shape, target.shape)
        
        # Inference
        output = model(input)
        
        #Save
        np_array = output.squeeze()[:, : H, : W].permute(1,2,0).cpu().numpy() * 255
        np_array = cv2.cvtColor(np_array, cv2.COLOR_BGR2RGB)
        cv2.imwrite(f"{output_path}/{i:02d}.jpeg", np_array)
        

In [71]:
inference_model(task="denoise", metric="psnr", mode="max")

# for idx in indexes:
    
#     plt.imshow(dataset[idx][0])
#     plt.show()

Found model with best psnr value 25.3820


In [6]:
metrics = [
    "psnr", 
    "ssim",
    "ms-ssim",
    "ms-gmsdc",
    "fsimc",
    "vsi",
    "mdsi",
#     "vifp",
    "content_vgg16_ap",
    "style_vgg16",
    "lpips",
    "dists",
    "brisque",
#     "is_metric_vgg16",
#     "is_vgg16",
    "kid_vgg16",
    "fid_vgg16",
    "msid_vgg16"
]

MODE_FROM_NAME = {
    # Full Reference
    "mae": "min",
    "mse": "min",
    "psnr": "max",
    "psnr_y": "max",
    "ssim": "max",
    "ms-ssim": "max",
    "vifp": "max",
    "vifp_2": "max",
    "gmsd": "min",
    "ms-gmsd": "min",
    "ms-gmsdc": "min",
    "fsim": "max",
    "fsimc": "max",
    "vsi": "max",
    "mdsi": "max",

    "content_vgg16": "min",
    "content_vgg16_ap": "min",

    "style_vgg16": "min",
    
    "lpips": "min",
    "dists": "min",

    # No reference
    "brisque": "min",

    # Distribution based metrics
    "fid_vgg16": "min",
    "kid_vgg16": "min",
    "gs_vgg16": "min",
    "is_metric_vgg16": "min",
    "is_vgg16": "min",
    "msid_vgg16": "min",
}

tasks = [
#     "denoise",
#     "deblur",
    "sr"
]

In [73]:
for task in tasks:
    for metric in metrics:
        inference_model(task=task, metric=metric, mode=MODE_FROM_NAME[metric])
        
    

Found model with best psnr value 25.5752
Found model with best ssim value 0.7910
Found model with best ms-ssim value 0.9667
Found model with best ms-gmsdc value 0.0583
Found model with best fsimc value 0.8802
Found model with best vsi value 0.9622
Found model with best mdsi value 0.5007
Found model with best vifp value 0.7761
Found model with best content_vgg16_ap value 0.1685
Found model with best style_vgg16 value 157959824.0000
Found model with best lpips value 0.1944
Found model with best dists value 0.1459
Found model with best brisque value 41.2568
Found model with best is_metric_vgg16 value 1.1299
Found model with best is_vgg16 value 0.5701
Found model with best kid_vgg16 value 0.0191
Found model with best fid_vgg16 value 14.2605
Found model with best msid_vgg16 value 11.5287


In [7]:
NUM_METRICS = len(metrics)
NUM_IMAGES = len(INDICES)
NUM_TASKS = 3  # SR, Deblur, Denoise
print(NUM_METRICS, NUM_IMAGES, NUM_TASKS)

15 15 3


In [41]:
# Create JSON file for label studio
# 1. All possible permutation pairs for all images and tasks
# 2. Total number: C^NUM_METRICS_2 * NUM_IMAGES * NUM_TASKS ~ 4700 for 15 metrics, 15 images and 3 tasks
import itertools
PORT = 6113
NGROK = "http://38b5b1a77e0b.eu.ngrok.io"
LEFT_LABEL = "left_img"
RIGHT_LABEL = "right_img"


ROOT = "data/images"
def create_json(task="deblur", image_id=0, metrics_list=["psnr", "ssim", "ms-ssim"]):
    # Read all relevant files
    path = os.path.join("data/processed", task)

    files = []
    for metric in metrics_list:
        file = os.path.join(path, metric, f"{image_id:02d}.jpeg")
        files.append(file)
        
    result = []
    # Create tasks file
    for (left, right) in itertools.combinations(files, 2):
        d = {
#             LEFT_LABEL: os.path.join(NGROK, left),
            LEFT_LABEL: os.path.abspath(left),
            RIGHT_LABEL: os.path.join(NGROK, right),
            "task": task,
            "left_metric": left.split("/")[3],
            "right_metric": right.split("/")[3],
            "image_id": image_id,
        }
        result.append(d)
    return result
        
    
#     print(list(combinations))

    
# for task in tasks:
#     pass
result = create_json()
with open("data/tasks.json", "w") as f:
    json.dump(result, f)
print(result)

[{'left_img': '/home/zakirov/repoz/metrics-comparison/data/processed/deblur/psnr/00.jpeg', 'right_img': 'http://38b5b1a77e0b.eu.ngrok.io/data/processed/deblur/ssim/00.jpeg', 'task': 'deblur', 'left_metric': 'psnr', 'right_metric': 'ssim', 'image_id': 0}, {'left_img': '/home/zakirov/repoz/metrics-comparison/data/processed/deblur/psnr/00.jpeg', 'right_img': 'http://38b5b1a77e0b.eu.ngrok.io/data/processed/deblur/ms-ssim/00.jpeg', 'task': 'deblur', 'left_metric': 'psnr', 'right_metric': 'ms-ssim', 'image_id': 0}, {'left_img': '/home/zakirov/repoz/metrics-comparison/data/processed/deblur/ssim/00.jpeg', 'right_img': 'http://38b5b1a77e0b.eu.ngrok.io/data/processed/deblur/ms-ssim/00.jpeg', 'task': 'deblur', 'left_metric': 'ssim', 'right_metric': 'ms-ssim', 'image_id': 0}]


In [9]:
data = {
  "0": {
    "id": 0,
    "data": {
      "image": "http://localhost:8080/data/upload/d68e7813b0a6e4746757e065c3230fcf-16.jpeg"
    }
  },
  "1": {
    "id": 1,
    "data": {
      "image": "http://localhost:8080/data/upload/ef189674482810635345c9b0cba26dbb-17.jpeg"
    }
  },
  "2": {
    "id": 2,
    "data": {
      "image": "http://localhost:8080/data/upload/54b632673940f9198546683d6aa3eee7-18.jpeg"
    }
  }
}

import json

In [13]:
j = json.dumps(data)
j

'{"0": {"id": 0, "data": {"image": "http://localhost:8080/data/upload/d68e7813b0a6e4746757e065c3230fcf-16.jpeg"}}, "1": {"id": 1, "data": {"image": "http://localhost:8080/data/upload/ef189674482810635345c9b0cba26dbb-17.jpeg"}}, "2": {"id": 2, "data": {"image": "http://localhost:8080/data/upload/54b632673940f9198546683d6aa3eee7-18.jpeg"}}}'

In [84]:
print(len(metrics))

tasks = 3

images = 20

18


In [31]:
from src.data import crop_patches

target = torch.rand(4, 3, 128, 128)
target_patches = crop_patches(target, size=96, stride=32)
print(target_patches.shape)

dataset = torch.utils.data.TensorDataset(target_patches)
print(dataset[0][0].shape)

torch.Size([16, 3, 96, 96])
torch.Size([3, 96, 96])


In [26]:
class PSNR(torch.nn.modules.loss._Loss):
    def __init(self, data_range=1.0, reduction='mean', convert_to_greyscale: bool = False):
        super().__init__()
        
        self.metric = functools.partial(
            piq.psnr, data_range=data_range, reduction=reduction, convert_to_greyscale=convert_to_greyscale)

    def forward(self, prediction: torch.Tensor, target: torch.Tensor):
        self.metric(prediction, target)


kwargs = {'data_range': 1.0, 'convert_to_greyscale': False},
PSNR(kwargs)

PSNR()

In [24]:
issubclass(piq.GMSDLoss, torch.nn.modules.loss._Loss)
issubclass(piq.fsim, torch.nn.modules.loss._Loss)

TypeError: issubclass() arg 1 must be a class

In [45]:
a = torch.rand(2, 3, 4, 4, 2)
res = torch.split(a, split_size_or_sections=1, dim=4)
res[1].shape

torch.Size([2, 3, 4, 4, 1])

In [15]:
piq.multi_scale_ssim?

In [13]:
prediction = torch.rand(4, 3, 128, 128)
piq.MultiScaleGMSDLoss()(prediction, prediction)

tensor(0.)

In [4]:
piq.ContentLoss?

In [2]:
a = "model_{ep}_{metric:.2f}.chpn"
a.format(ep=3, metric=0.234556436)

'model_3_0.23.chpn'

In [12]:
kwargs = {
    "feature_extractor": 'vgg19',
    "layers": ['conv1_2', 'conv2_2', 'conv3_4', 'conv4_4', 'conv5_4'],
    "weights": [0.2, 0.2, 0.2, 0.2, 0.2],
    "normalize_features": True,
    "reduction": 'none',
}

kwargs.pop("feature_extractor")
kwargs

{'layers': ['conv1_2', 'conv2_2', 'conv3_4', 'conv4_4', 'conv5_4'],
 'weights': [0.2, 0.2, 0.2, 0.2, 0.2],
 'normalize_features': True,
 'reduction': 'none'}

In [ ]:
        # DummyAverageMeter is added to the end, so just delete last part
        counter = 0
        for i, metric in enumerate(self.state.metric_meters):
            if isinstance(metric, AverageMeter):
                metric.reset()
            else:
                counter += 1
        self.state.metric_meters = self.state.metric_meters[:-counter]

In [67]:
reduction = 'none'
f = functools.partial(piq.vif_p, reduction=reduction)
f.name = "test"
print(f, f.name)

functools.partial(<function vif_p at 0x7f59f1780d08>, reduction='none') test


In [62]:
class A():
    def reset(self):
        print("A reseted")


class B():
    def reset(self):
        print("B reseted")

class State:
    def __init__(self, metric_meters):
        self.metric_meters = metric_meters

metric_meters = [A(), A(), B(), B(), B(), A()]
print(metric_meters)
state = State(metric_meters)

state.metric_meters = [m for m in state.metric_meters if isinstance(m, A)]
for metric in state.metric_meters:
    metric.reset()

# omelist = [x for x in somelist if not determine(x)]

# for i, metric in enumerate(state.metric_meters):
#     if isinstance(metric, A):
#         metric.reset()
#     elif isinstance(metric, B):
#         print("Deleting B")
#         del state.metric_meters[i]
print(state.metric_meters)
        
    

[<__main__.A object at 0x7f59d08592e8>, <__main__.A object at 0x7f59d0859860>, <__main__.B object at 0x7f59d08597f0>, <__main__.B object at 0x7f59d08598d0>, <__main__.B object at 0x7f59d0859898>, <__main__.A object at 0x7f59d0859828>]
A reseted
A reseted
A reseted
[<__main__.A object at 0x7f59d08592e8>, <__main__.A object at 0x7f59d0859860>, <__main__.A object at 0x7f59d0859828>]


In [51]:
import functools
METRIC_FROM_NAME = {
    # Full Reference
    "psnr": functools.partial(
        piq.psnr),
}

def get_metric(name, reduction='none'):
    metric = METRIC_FROM_NAME[name]
    return functools.partial(metric, reduction=reduction)
    
prediction = torch.rand(4, 3, 128, 128)
target = torch.rand(4, 3, 128, 128)

get_metric("psnr")(prediction, target)

tensor(7.7687)

In [55]:
a = [piq.ContentLoss(), piq.StyleLoss(), piq.LPIPS(), piq.DISTS(), piq.GMSDLoss()]

In [ ]:
[07-24 20:48] - Best loss: 0.0394
[07-24 20:48] - Best psnr: 25.0806
[07-24 20:48] - Best ssim: 0.7546
[07-24 20:48] - Best ms-ssim: 0.9332
[07-24 20:48] - Best gmsd: 0.0846
[07-24 20:48] - Best ms-gmsd: 0.0816
[07-24 20:48] - Best ms-gmsdc: 0.0681
[07-24 20:48] - Best fsim: 0.8802
[07-24 20:48] - Best fsimc: 0.8744
[07-24 20:48] - Best vsi: 0.9544
[07-24 20:48] - Best mdsi: 0.4146
[07-24 20:48] - Best vifp: 0.9085
[07-24 20:48] - Best content_vgg16: 0.2474
[07-24 20:48] - Best content_vgg19: 0.2679
[07-24 20:48] - Best content_vgg16_ap: 0.2176
[07-24 20:48] - Best content_vgg19_ap: 0.2257
[07-24 20:48] - Best style_vgg16: 9222874.0769
[07-24 20:48] - Best style_vgg19: 7961090.4615
[07-24 20:48] - Best lpips: 0.2275
[07-24 20:48] - Best dists: 0.1868
[07-24 20:48] - Best brisque: 37.7652

In [ ]:
[07-27 15:22] - Best loss: 0.0353
[07-27 15:22] - Best psnr: 25.4831
[07-27 15:22] - Best ssim: 0.7822
[07-27 15:22] - Best ms-ssim: 0.9491
[07-27 15:22] - Best gmsd: 0.0853
[07-27 15:22] - Best ms-gmsd: 0.0827
[07-27 15:22] - Best ms-gmsdc: 0.0693
[07-27 15:22] - Best fsim: 0.8753
[07-27 15:22] - Best fsimc: 0.8733
[07-27 15:22] - Best vsi: 0.9596
[07-27 15:22] - Best mdsi: 0.4467
[07-27 15:22] - Best vifp: 0.8642
[07-27 15:22] - Best content_vgg16_ap: 0.1814
[07-27 15:22] - Best style_vgg16: 147372154.8571
[07-27 15:22] - Best lpips: 0.2060
[07-27 15:22] - Best dists: 0.1557
[07-27 15:22] - Best brisque: 45.1469
[07-27 15:22] - Best is_metric_vgg16: 1.1791
[07-27 15:22] - Best is_vgg16: 0.4978
[07-27 15:22] - Best kid_vgg16: 0.0180
[07-27 15:22] - Best fid_vgg16: 14.8280
[07-27 15:22] - Best msid_vgg16: 8.8804

In [ ]:
[07-27 16:27] - Best loss: 0.0036
[07-27 16:27] - Best psnr: 25.3338
[07-27 16:27] - Best ssim: 0.7586
[07-27 16:27] - Best ms-ssim: 0.9371
[07-27 16:27] - Best gmsd: 0.0873
[07-27 16:27] - Best ms-gmsd: 0.0832
[07-27 16:27] - Best ms-gmsdc: 0.0696
[07-27 16:27] - Best fsim: 0.8803
[07-27 16:27] - Best fsimc: 0.8775
[07-27 16:27] - Best vsi: 0.9612
[07-27 16:27] - Best mdsi: 0.4124
[07-27 16:27] - Best vifp: 0.9856
[07-27 16:27] - Best content_vgg16_ap: 0.1994
[07-27 16:27] - Best style_vgg16: 56819345.4286
[07-27 16:27] - Best lpips: 0.2112
[07-27 16:27] - Best dists: 0.1431
[07-27 16:27] - Best brisque: 36.5921
[07-27 16:27] - Best is_metric_vgg16: 1.2533
[07-27 16:27] - Best is_vgg16: 0.3236
[07-27 16:27] - Best kid_vgg16: 0.0104
[07-27 16:27] - Best fid_vgg16: 11.7871
[07-27 16:27] - Best msid_vgg16: 4.0915
[07-27 16:27] - Finished training!

In [ ]:
[07-27 17:22] - Best loss: 0.0567
[07-27 17:22] - Best psnr: 24.6242
[07-27 17:22] - Best ssim: 0.7714
[07-27 17:22] - Best ms-ssim: 0.9410
[07-27 17:22] - Best gmsd: 0.0834
[07-27 17:22] - Best ms-gmsd: 0.0799
[07-27 17:22] - Best ms-gmsdc: 0.0663
[07-27 17:22] - Best fsim: 0.8866
[07-27 17:22] - Best fsimc: 0.8804
[07-27 17:22] - Best vsi: 0.9596
[07-27 17:22] - Best mdsi: 0.4374
[07-27 17:22] - Best vifp: 0.9039
[07-27 17:22] - Best content_vgg16_ap: 0.2017
[07-27 17:22] - Best style_vgg16: 290767860.0000
[07-27 17:22] - Best lpips: 0.2258
[07-27 17:22] - Best dists: 0.1545
[07-27 17:22] - Best brisque: 34.5464
[07-27 17:22] - Best is_metric_vgg16: 1.3126
[07-27 17:22] - Best is_vgg16: 0.3683
[07-27 17:22] - Best kid_vgg16: 0.0117
[07-27 17:22] - Best fid_vgg16: 13.4920
[07-27 17:22] - Best msid_vgg16: 7.6096
[07-27 17:22] - Finished training!